## Computing daily averages from transaction data - Part 2

In Part 1 , we discuss the importance of taking into account days with zero activity as well as carefully specifiying the date range for analysis when computing things like the average number of bike trips by day in a bike share system. Obviously, this applies more generally to computing averages number of events per day. We showed an approach to dealing with these complications.

Now we'll extend this to cases in which we might want to group by an additional field (e.g. station_id for the bike share example) and see how this problem is just one part of the general problem of doing occupancy analysis based on transaction data. We'll see how the [hillmaker](https://github.com/misken/hillmaker) package can make these types of analyses easier.

In [ ]:
import pandas as pd

In [ ]:
trip = pd.read_csv('data/trip.csv', parse_dates = ['starttime', 'stoptime'])
trip['tripdate'] = trip['starttime'].dt.date

In [ ]:
trip.info()

In [ ]:
trip.head()

# What is the average number of bike rentals per day from each station?

Not only do we need to take into account days in which no bikes were rented from each of the stations, we have to decide how we want to handle the fact that not all stations were open for the period of time represented in the `trip` dataframe. 

In [ ]:
print(f"The first trip was at {trip.starttime.min()}.")
print(f"The last trip was at {trip.starttime.max()}.")

We can get a sense of the timeframes for when individual stations were open by looking an min and max of the `starttime` field.

In [ ]:
trip.groupby('from_station_id')['starttime'].aggregate(['min', 'max'])

Let's look at the `station.csv` file so that we can see the installation and decommision dates.

In [ ]:
station = pd.read_csv('data/station.csv', 
                      parse_dates = ['install_date', 'modification_date', 'decommission_date'])

In [ ]:
station.head(15)

To simplify things, let's only consider trips for stations who have **not** been decommissioned.

In [ ]:
# Create a list of stations to consider for the analysis
active_stations = list(station[station.decommission_date.isna()]['station_id'])
# Only keep records for active stations
trip = trip[trip.from_station_id.isin(active_stations)]

Now let's create a DataFrame containing the analysis start and end dates for each active stations. For the start date we'll use the `install_date` field and for end date we'll use the last date in the trip data - 2016-08-31.

In [ ]:
# Just grab the rows of active stations
station_dates = station[station.decommission_date.isna()]
# Just grab columns of interest
station_dates = station_dates[['station_id','install_date','decommission_date']]
# Rename the date columns
station_dates.rename(columns={'install_date': 'start_date', 'decommission_date': 'end_date'},
                    inplace=True)
# Set end date for all stations
station_dates.end_date = pd.Timestamp(2016, 8, 31)
# Check out the result
station_dates.head(7)

In Part 1 we outlined the basic strategy for computing the average number of daily events when some days might not have any events. 

1. Create a range of dates based on the start and end date of the time period of interest.
2. Create an empty DataFrame using the range of dates as the index. For convenience, add weekday column based on date to facility day of week analysis. Let's call this new DataFrame the "seeded" DataFrame
3. Use groupby on the original trip DataFrame to compute number of trips by date and store result as DataFrame
4. Merge the two DataFrames on their indexes (tripdate) but do a "left join". Pandas `merge` function is perfect for this.
5. If there are dates with no trips, they'll have missing data in the new merged DataFrame. Update the missing values to 0 using the `fillna` function in pandas.
6. Now you can compute overall mean number of trips per day. You could also compute means by day of week.

In order to compute the average number of trips per day by station, a few details in the above process have to be generalized.

* In Step 1, the start and end dates can depend on the station.
* In Step 2, we need a DataFrame containing each station and its associated date range.
* In Step 3, we need to group by both station and trip date.


## Accounting for zero days and station specific analysis timeframe

### Steps 1 and 2: Create an empty DataFrame using stations and associated range of dates as the MultiIndex.

We'll construct the desired MultiIndex from a list of tuples of station ids and dates. To create the list of tuples we can use a list comprehension to iterate over the rows of the `station_dates` dataframe and associated date range. Gotta love list comprehensions.

In [ ]:
index_tuples = [(row.station_id, d) for index, row in station_dates.iterrows() 
                         for d in pd.date_range(row.start_date, row.end_date)]

print(index_tuples[:5])

Create the empty DataFrame and add a weekday column for convenience.

In [ ]:
trips_by_station_date_seeded = pd.DataFrame(index=pd.MultiIndex.from_tuples(index_tuples))

In [ ]:
# Add weekday column to new dataframe
trips_by_station_date_seeded['weekday'] = \
    trips_by_station_date_seeded.index.get_level_values(1).map(lambda x: x.weekday())

trips_by_station_date_seeded.index.names = ['from_station_id', 'tripdate']
print(trips_by_station_date_seeded.head())
print(trips_by_station_date_seeded.tail())

### Step 3: Use groupby on the original trip DataFrame to compute number of trips by station by date 

In [ ]:
# Create a Group by object using from_station_id and tripdate
grp_station_date = trip.groupby(['from_station_id', 'tripdate'])

# Compute number of trips by date and check out the result
trips_by_station_date = pd.DataFrame(grp_station_date.size(), columns=['num_trips'])
print(trips_by_station_date.head())

### Step 4: Merge the two DataFrames on their indexes but do a "left join". 

Pandas `merge` function is perfect for this. The `left_index=True, right_index=True` are telling pandas to use those respective indexes
as the joining columns. Check out the [documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) for `merge()`.

In [ ]:
# Merge the two dataframes doing a left join (with seeded table on left)
trips_by_station_date_merged = pd.merge(trips_by_station_date_seeded, 
                                        trips_by_station_date, how='left', 
                                        left_index=True, right_index=True, sort=True)

In [ ]:
trips_by_station_date_merged.head()

### Step 5: Replace missing values with zeroes for those dates with no trips

If there are dates with no trips, they'll have missing data in the new merged DataFrame. Update the missing values to 0 using the `fillna` function in pandas. Remember, these are the instances of "zero activity" that would cause biased (high) estimates of the average number of daily trips.

In [ ]:
# How many missing values do we have?
trips_by_station_date_merged['num_trips'].isna().sum()

In [ ]:
# Fill in any missing values with 0.
trips_by_station_date_merged['num_trips'] = trips_by_station_date_merged['num_trips'].fillna(0)

### Step 6: Compute statistics of interest

Now we can safely compute means and other statistics of interest for the `num_trips` column after grouping by `from_station_id`. Since we have taken the zero activity days into account, the stations having count values less than 689 are those stations which were installed sometime after 2014-10-13 (the installation date is in the `station` table).

In [ ]:
trips_by_station_date_merged.groupby('from_station_id')['num_trips'].describe()

Can even do by day of week.

In [ ]:
trips_by_station_date_merged.groupby(['from_station_id', 'weekday'])['num_trips'].describe()

## Closing remarks - check out hillmaker

In addition to simply counting number of bike rentals, we might be interested in related statistics such as the average and percentiles of the number of bikes in use by time of day and day of week. My [hillmaker](https://github.com/misken/hillmaker) package is designed for just such analysis. You can find an example here of it being used to [analyze the Pronto Cycle Share data](https://misken.github.io/blog/basic_usage_cycleshare/).